In [15]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pylab as plt
#%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

from scipy.stats import norm
from scipy import stats
from sklearn.impute import KNNImputer


from sklearn.model_selection import GridSearchCV, cross_val_score

In [3]:
#loading the data from CSV file 
data=pd.read_csv('finalDataset.csv')


In [4]:
# Create a MinMaxScaler object for numrical data
scaler = MinMaxScaler()

# Scaling the raw input features 
feature_cols=data.columns[:-1]
X= scaler.fit_transform(data[feature_cols])

print(f"The range of feature inputs are within {X.min()} to {X.max()}")

The range of feature inputs are within 0.0 to 1.0


In [5]:

from sklearn.model_selection import StratifiedShuffleSplit

# Get the split indexes
strat_shuf_split = StratifiedShuffleSplit(n_splits=1, 
                                          test_size=0.3, random_state=0)

train_idx, test_idx = next(strat_shuf_split.split(data[feature_cols], data['Rate']))

# Create the dataframes
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'Rate']

X_test  = data.loc[test_idx, feature_cols]
y_test  = data.loc[test_idx, 'Rate']

print(f"Training dataset shape, X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Testing dataset shape, X_test: {X_test.shape}, y_test: {y_test.shape}")

Training dataset shape, X_train: (217, 19), y_train: (217,)
Testing dataset shape, X_test: (94, 19), y_test: (94,)


In [6]:
y_train.value_counts(normalize=True)

5    0.718894
4    0.161290
3    0.092166
1    0.009217
0    0.009217
2    0.009217
Name: Rate, dtype: float64

In [7]:
y_test.value_counts(normalize=True)

5    0.723404
4    0.159574
3    0.085106
0    0.010638
2    0.010638
1    0.010638
Name: Rate, dtype: float64

In [19]:
# DecisionTreeClassifier
# importing the classfier



def DecisionTree(X_train , X_test, y_train , y_test ):
    clf=DecisionTreeClassifier(random_state=0)
    clf=clf.fit(X_train,Y_train)

    #optimization
    param_grid={
    "max_depth":[2,4,6],
    "min_samples_split":[2,5,10],
    "min_samples_leaf":[1,2,4]}
    grid_search= GridSearchCV(estimator=clf,param_grid=param_grid,cv=5)

    grid_search.fit(X_train,Y_train)
    print("Best hyper-param: ",grid_search.best_params_ )
    print("Best estimator: ",grid_search.best_estimator_ )
    print("Best score: ",grid_search.best_score_ )

    #report 
    print(classification_report(Y_test,pred,target_names=['0','1','2','3','4','5']))

    #confusion_matrix
    #the result will show how mwny sucessful predition and wrong from each class

    cm = confusion_matrix(Y_test, pred)
    plt.figure(figsize=(10,7))

    sns.heatmap(cm, annot=True, fmt='.2g', cmap='Blues')

    # TN   FP
    # FN   TP





In [17]:
#optimization
param_grid={
    "max_depth":[2,4,6],
    "min_samples_split":[2,5,10],
    "min_samples_leaf":[1,2,4]}
grid_search= GridSearchCV(estimator=clf,param_grid=param_grid,cv=5)

grid_search.fit(X_train,Y_train)
print("Best hyper-param: ",grid_search.best_params_ )
print("Best estimator: ",grid_search.best_estimator_ )
print("Best score: ",grid_search.best_score_ )

NameError: name 'clf' is not defined

In [ ]:
#report 
print(classification_report(Y_test,pred,target_names=['0','1','2','3','4','5']))